# 🍓 Strawberry Vision - Google Colab Eğitim Pipeline

Bu notebook, Roboflow'dan dataset indirip YOLOv8 ile çilek tespit ve olgunluk sınıflandırma modeli eğitmenizi sağlar.

## 📋 Pipeline Adımları
1. **Kurulum** - Bağımlılıklar ve proje yapısı
2. **Roboflow Dataset** - API ile otomatik indirme
3. **Sınıf Etiketleme** - Otomatik standardizasyon
4. **Model Eğitimi** - YOLOv8 eğitimi
5. **Inference** - Eğitilmiş model ile tahmin
6. **Sonuçları Kaydet** - Model ve raporları indir

## ⚡ Hızlı Başlangıç
1. Runtime > Change runtime type > **GPU** seçin
2. Tüm hücreleri sırayla çalıştırın
3. Roboflow API key'inizi girin (Bölüm 2)

---

## 📦 Bölüm 1: Kurulum ve Proje Yapısı

In [ ]:
# 1.1. Bağımlılıkları Yükle
print("📦 Bağımlılıklar yükleniyor...")

!pip install -q ultralytics>=8.0.0
!pip install -q opencv-python>=4.8.0
!pip install -q numpy>=1.24.0
!pip install -q matplotlib>=3.7.0
!pip install -q roboflow>=1.1.0
!pip install -q pyyaml>=6.0

print("✅ Kurulum tamamlandı!")

In [ ]:
# 1.2. Proje Klasörlerini Oluştur
import os
from pathlib import Path

# Gerekli klasörler
folders = ['datasets', 'runs', 'models']
for folder in folders:
    Path(folder).mkdir(exist_ok=True)
    print(f"✅ {folder}/")

print("\n📁 Proje yapısı hazır!")

## 🌐 Bölüm 2: Roboflow Dataset İndirme

### Önerilen Datasetler:
1. **Strawberry Ripeness Classification** - `fruit-detection/strawberry-ripeness`
2. **Strawberry Detection Dataset** - `strawberry-detection/strawberry-detection-dataset`
3. **Agricultural Strawberry** - `agriculture/strawberry-field-detection`

### API Key Alma:
https://app.roboflow.com/settings/api

In [ ]:
# 2.1. Roboflow API Ayarları
from roboflow import Roboflow

# ⚠️ API KEY'İNİZİ BURAYA GİRİN
API_KEY = "YOUR_API_KEY_HERE"  # https://app.roboflow.com/settings/api

# Dataset seçimi (aşağıdakilerden birini seçin)
WORKSPACE = "fruit-detection"  # veya "strawberry-detection" veya "agriculture"
PROJECT = "strawberry-ripeness"  # veya "strawberry-detection-dataset" veya "strawberry-field-detection"
VERSION = 1

print(f"📥 Dataset: {WORKSPACE}/{PROJECT} v{VERSION}")

In [ ]:
# 2.2. Dataset İndir
if API_KEY == "YOUR_API_KEY_HERE":
    print("⚠️  Lütfen API KEY'inizi girin!")
    print("API Key almak için: https://app.roboflow.com/settings/api")
else:
    print("🔄 Roboflow'dan indiriliyor...")
    
    rf = Roboflow(api_key=API_KEY)
    project = rf.workspace(WORKSPACE).project(PROJECT)
    dataset = project.version(VERSION).download("yolov8", location="datasets/roboflow")
    
    print("✅ Dataset indirildi: datasets/roboflow")
    print(f"📊 Dataset bilgileri: {dataset.location}")

## 🔄 Bölüm 3: Sınıf Etiketlerini Standardize Et

Roboflow'dan gelen sınıf isimleri farklı olabilir. Standart formata çeviriyoruz:
- `strawberry_ripe` - Olgun çilek
- `strawberry_semi_ripe` - Yarı olgun çilek  
- `strawberry_unripe` - Olgun olmayan çilek

In [ ]:
# 3.1. Sınıf Mapping Tanımla
import yaml
import shutil

CLASS_MAPPING = {
    "ripe": "strawberry_ripe",
    "semi-ripe": "strawberry_semi_ripe",
    "semi_ripe": "strawberry_semi_ripe",
    "unripe": "strawberry_unripe",
    "green": "strawberry_unripe",
    "strawberry": "strawberry_ripe",
}

STANDARD_CLASSES = {
    0: "strawberry_ripe",
    1: "strawberry_semi_ripe",
    2: "strawberry_unripe",
}

print("✅ Sınıf mapping hazır")

In [ ]:
# 3.2. data.yaml Oku ve Güncelle
yaml_path = "datasets/roboflow/data.yaml"

if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    old_names = data_config.get('names', [])
    print(f"Eski sınıflar: {old_names}")
    
    # ID mapping oluştur
    if isinstance(old_names, list):
        old_names = {i: name for i, name in enumerate(old_names)}
    
    id_mapping = {}
    for old_id, old_name in old_names.items():
        mapped_name = CLASS_MAPPING.get(old_name.lower(), "strawberry_ripe")
        for new_id, new_name in STANDARD_CLASSES.items():
            if mapped_name == new_name:
                id_mapping[old_id] = new_id
                break
    
    print(f"ID Mapping: {id_mapping}")
else:
    print("⚠️  data.yaml bulunamadı")

In [ ]:
# 3.3. Label Dosyalarını Güncelle
from pathlib import Path

def update_labels(labels_dir, id_mapping):
    """Label dosyalarındaki class ID'leri güncelle"""
    updated_count = 0
    
    for label_file in Path(labels_dir).glob('*.txt'):
        with open(label_file, 'r') as f:
            lines = f.readlines()
        
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                old_class_id = int(parts[0])
                new_class_id = id_mapping.get(old_class_id, old_class_id)
                parts[0] = str(new_class_id)
                new_lines.append(' '.join(parts) + '\n')
                if new_class_id != old_class_id:
                    updated_count += 1
        
        with open(label_file, 'w') as f:
            f.writelines(new_lines)
    
    return updated_count

# Train, val, test labellarını güncelle
total_updated = 0
for split in ['train', 'valid', 'test']:
    labels_dir = f"datasets/roboflow/{split}/labels"
    if os.path.exists(labels_dir):
        count = update_labels(labels_dir, id_mapping)
        total_updated += count
        print(f"✅ {split}: {count} etiket güncellendi")

print(f"\n📊 Toplam {total_updated} etiket güncellendi")

In [ ]:
# 3.4. Yeni data.yaml Oluştur
new_data_yaml = {
    'path': os.path.abspath('datasets/roboflow'),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 3,
    'names': STANDARD_CLASSES
}

with open('datasets/roboflow/data.yaml', 'w') as f:
    yaml.dump(new_data_yaml, f, default_flow_style=False)

print("✅ data.yaml güncellendi")
print(f"Sınıflar: {STANDARD_CLASSES}")

## 🎓 Bölüm 4: YOLOv8 Model Eğitimi

### Önerilen Parametreler:
- **Hızlı Test**: yolov8n.pt, 20 epoch, batch 16
- **Orta Kalite**: yolov8s.pt, 50 epoch, batch 16  
- **Yüksek Kalite**: yolov8m.pt, 100 epoch, batch 8-16

In [ ]:
# 4.1. Eğitim Parametreleri
TRAIN_CONFIG = {
    'model': 'yolov8n.pt',      # Model boyutu (n/s/m/l/x)
    'epochs': 50,                # Epoch sayısı
    'batch': 16,                 # Batch size (GPU'ya göre ayarlayın)
    'imgsz': 640,                # Görüntü boyutu
    'device': 0,                 # GPU (0) veya CPU ('cpu')
    'project': 'runs/train',     # Sonuç klasörü
    'name': 'strawberry_exp',    # Deney adı
    'patience': 50,              # Early stopping
    'save_period': 10,           # Checkpoint kaydetme
    'optimizer': 'AdamW',
    'lr0': 0.01,
    'lrf': 0.01,
    'hsv_h': 0.015,              # Augmentation
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 10.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,
}

print("🎯 Eğitim Parametreleri:")
for key, value in TRAIN_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# 4.2. Model Eğitimi Başlat
from ultralytics import YOLO

print("🚀 Model eğitimi başlıyor...")
print("⏱️  Bu işlem 30-60 dakika sürebilir (GPU ile)\n")

# Model yükle
model = YOLO(TRAIN_CONFIG['model'])

# Eğitim başlat
results = model.train(
    data='datasets/roboflow/data.yaml',
    **TRAIN_CONFIG
)

print("\n✅ Eğitim tamamlandı!")
print(f"📊 Sonuçlar: {TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}")
print(f"🏆 En iyi model: {TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}/weights/best.pt")

In [ ]:
# 4.3. Eğitim Sonuçlarını Görselleştir
from IPython.display import Image, display
import matplotlib.pyplot as plt

results_path = f"{TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}"

print("📊 Eğitim Grafikleri:\n")

# Results
if os.path.exists(f"{results_path}/results.png"):
    display(Image(filename=f"{results_path}/results.png"))

# Confusion Matrix
if os.path.exists(f"{results_path}/confusion_matrix.png"):
    display(Image(filename=f"{results_path}/confusion_matrix.png"))

# Validation Predictions
if os.path.exists(f"{results_path}/val_batch0_pred.jpg"):
    display(Image(filename=f"{results_path}/val_batch0_pred.jpg"))

## 🔍 Bölüm 5: Model Değerlendirme ve Inference

In [ ]:
# 5.1. Model Değerlendirme
best_model_path = f"{TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}/weights/best.pt"

if os.path.exists(best_model_path):
    print("📊 Model değerlendiriliyor...\n")
    
    model = YOLO(best_model_path)
    metrics = model.val(data='datasets/roboflow/data.yaml')
    
    print("\n✅ Değerlendirme Sonuçları:")
    print(f"  mAP@0.5: {metrics.box.map50:.4f}")
    print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall: {metrics.box.mr:.4f}")
else:
    print("⚠️  Model bulunamadı")

In [ ]:
# 5.2. Test Görüntüsü ile Inference
import cv2
import numpy as np

# Test görüntüsü seç (validation setinden)
test_images_dir = "datasets/roboflow/valid/images"
test_images = list(Path(test_images_dir).glob('*.jpg'))[:5]  # İlk 5 görüntü

if test_images and os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    for img_path in test_images:
        # Tahmin yap
        results = model(str(img_path))
        
        # Sonuçları görselleştir
        annotated = results[0].plot()
        
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Tespit: {img_path.name}")
        plt.show()
        
        # Tespit sayısı
        boxes = results[0].boxes
        print(f"\n{img_path.name}: {len(boxes)} çilek tespit edildi")
        
        # Sınıf dağılımı
        if len(boxes) > 0:
            class_counts = {}
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = STANDARD_CLASSES[cls_id]
                class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
            print(f"Dağılım: {class_counts}")
        print("-" * 50)
else:
    print("⚠️  Test görüntüleri veya model bulunamadı")

## 💾 Bölüm 6: Sonuçları Kaydet ve İndir

In [ ]:
# 6.1. Sonuçları Zip'le
import shutil
from google.colab import files

results_dir = f"{TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}"

if os.path.exists(results_dir):
    print("📦 Sonuçlar zip'leniyor...")
    
    zip_name = f"strawberry_model_{TRAIN_CONFIG['name']}"
    shutil.make_archive(zip_name, 'zip', results_dir)
    
    print(f"✅ Zip oluşturuldu: {zip_name}.zip")
    print(f"📊 Boyut: {os.path.getsize(f'{zip_name}.zip') / (1024*1024):.2f} MB")
    
    # İndir
    print("\n⬇️  İndirme başlıyor...")
    files.download(f"{zip_name}.zip")
    
    print("✅ İndirme tamamlandı!")
else:
    print("⚠️  Sonuç klasörü bulunamadı")

In [ ]:
# 6.2. Sadece En İyi Modeli İndir
if os.path.exists(best_model_path):
    print("🏆 En iyi model indiriliyor...")
    
    # Model dosyasını kopyala
    shutil.copy(best_model_path, 'strawberry_best.pt')
    
    files.download('strawberry_best.pt')
    
    print("✅ Model indirildi: strawberry_best.pt")
    print(f"📊 Boyut: {os.path.getsize('strawberry_best.pt') / (1024*1024):.2f} MB")
else:
    print("⚠️  Model bulunamadı")

## 📝 Özet ve Notlar

### ✅ Tamamlanan Adımlar
1. Roboflow'dan dataset indirildi
2. Sınıf etiketleri standardize edildi
3. YOLOv8 modeli eğitildi
4. Model değerlendirildi
5. Sonuçlar kaydedildi

### 📊 Performans Hedefleri
- mAP@0.5 ≥ %80
- Precision ≥ %85
- Recall ≥ %75

### 🔧 Sorun Giderme
- **Out of Memory**: `batch` değerini düşürün (16 → 8)
- **Yavaş Eğitim**: GPU'nun aktif olduğunu kontrol edin
- **Düşük Accuracy**: `epochs` artırın veya daha büyük model deneyin

### 📚 Kaynaklar
- [YOLOv8 Docs](https://docs.ultralytics.com)
- [Roboflow Universe](https://universe.roboflow.com)
- [Proje Dokümantasyonu](https://github.com/your-repo)

---

**🎉 Tebrikler!** Çilek tespit modeliniz hazır. Artık kendi görüntülerinizde kullanabilirsiniz.